In [2]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [34]:
df = pd.read_csv("marketing_AB.csv", index_col= 0)
df.head()

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14


Ejericicios de A/B Testing
Ejercicio 1:

   Objetivo del ejercicio: Comparar las tasas de conversión entre los grupos de prueba "ad" y "psa" para determinar si el nuevo diseño de anuncio es más efectivo en la conversión de usuarios.
        Definición de las hipótesis: 

            Hipótesis nula (H0): No hay diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".

            Hipótesis alternativa (H1): Existe una diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".


a) Asegúrate de que los datos estén limpios y listos para el análisis. Esto incluye la eliminación de datos faltantes

In [9]:
# una vez que tengamos nuestro DataFrame preparado con todas las columnas que queremos vamos a crear una función que no haga una exploración inicial del conjunto de datos
def exploracion_dataframe(dataframe, columna_control):
        # generamos un DataFrame para los valores nulos
    print("Los nulos que tenemos en el conjunto de datos son:")
    df_nulos = pd.DataFrame(dataframe.isnull().sum() / dataframe.shape[0] * 100, columns = ["%_nulos"])
    display(df_nulos[df_nulos["%_nulos"] > 0])
    
    print("\n ..................... \n")
    print(f"Los tipos de las columnas son:")
    display(pd.DataFrame(dataframe.dtypes, columns = ["tipo_dato"]))
    
    
    print("\n ..................... \n")
    print("Los valores que tenemos para las columnas categóricas son: ")
    dataframe_categoricas = dataframe.select_dtypes(include = "O")
    
    for col in dataframe_categoricas.columns:
        print(f"La columna {col.upper()} tiene las siguientes valore únicos:")
        display(pd.DataFrame(dataframe[col].value_counts()).head())    
    
    # como estamos en un problema de A/B testing y lo que realmente nos importa es comparar entre el grupo de control y el de test, los principales estadísticos los vamos a sacar de cada una de las categorías
    
    for categoria in dataframe[columna_control].unique():
        
        dataframe_filtrado = dataframe[dataframe[columna_control] == categoria]
    
        print("\n ..................... \n")
        print(f"Los principales estadísticos de las columnas categóricas para el {categoria.upper()} son: ")
        display(dataframe_filtrado.describe(include = "O").T)
        
        print("\n ..................... \n")
        print(f"Los principales estadísticos de las columnas numéricas para el {categoria.upper()} son: ")
        display(dataframe_filtrado.describe().T)

In [11]:
df['test group'].unique()

array(['ad', 'psa'], dtype=object)

In [12]:
df.isnull().sum()

user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64

In [13]:
df.duplicated().sum()

0

In [10]:
exploracion_dataframe(df,"test group")

Los nulos que tenemos en el conjunto de datos son:


,%_nulos



 ..................... 

Los tipos de las columnas son:


,tipo_dato
user id,int64
test group,object
converted,bool
total ads,int64
most ads day,object
most ads hour,int64



 ..................... 

Los valores que tenemos para las columnas categóricas son: 
La columna TEST GROUP tiene las siguientes valore únicos:


...

,count,mean,std,min,25%,50%,75%,max
user id,564577.0,1.327314e+06,188918.061566,1000000.0,1163686.0,1327362.0,1490914.0,1654483.0
total ads,564577.0,2.482337e+01,43.750456,1.0,4.0,13.0,27.0,2065.0
most ads hour,564577.0,1.447590e+01,4.841808,0.0,11.0,14.0,18.0,23.0



 ..................... 

Los principales estadísticos de las columnas categóricas para el PSA son: 


,count,unique,top,freq
test group,23524,1,psa,23524
most ads day,23524,7,Thursday,3905



 ..................... 

Los principales estadísticos de las columnas numéricas para el PSA son: 


,count,mean,std,min,25%,50%,75%,max
user id,23524.0,911761.500000,6790.938202,900000.0,905880.75,911761.5,917642.25,923523.0
total ads,23524.0,24.761138,42.860720,1.0,4.00,12.0,26.00,907.0
most ads hour,23524.0,14.304923,4.656239,0.0,11.00,14.0,18.00,23.0


b) Calculo de la tasa de conversion

c) verificación de la coherencia de los datos.

In [14]:
df.head()

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14


In [15]:
df['converted'].value_counts()

converted
False    573258
True      14843
Name: count, dtype: int64

In [16]:
df['test group'].value_counts()

test group
ad     564577
psa     23524
Name: count, dtype: int64

In [25]:
#Para el grupo "ad", la tasa de conversión es el número de usuarios convertidos dividido por el número total de usuarios en ese grupo.

# calculamos la Tasa de Conversión
ad_convertido = round ((14843 / 564577), 2)
ad_convertido

0.03

In [26]:
# calculamos la Tasa de Conversión
psa_convertido = round((14843 / 23524) , 2)
psa_convertido

0.63

In [20]:
df.head()

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14


In [21]:
stats.shapiro?

Signature: stats.shapiro(x, *, axis=None, nan_policy='propagate', keepdims=False)
Docstring:
Perform the Shapiro-Wilk test for normality.

The Shapiro-Wilk test tests the null hypothesis that the
data was drawn from a normal distribution.

Parameters
----------
x : array_like
    Array of sample data.
axis : int or None, default: None
    If an int, the axis of the input along which to compute the statistic.
    The statistic of each axis-slice (e.g. row) of the input will appear in a
    corresponding element of the output.
    If ``None``, the input will be raveled before computing the statistic.
nan_policy : {'propagate', 'omit', 'raise'}
    Defines how to handle input NaNs.
    
    - ``propagate``: if a NaN is present in the axis slice (e.g. row) along
      which the  statistic is computed, the corresponding entry of the output
      will be NaN.
    - ``omit``: NaNs will be omitted when performing the calculation.
      If insufficient data remains in the axis slice along which

# Desde aqui comienzo de nuevo considerando lo que dijeron las profres y ayuda de chat gpt

Ejercicio 1: determinar si un nuevo diseño de anuncio (test group "ad") tiene un impacto significativo en la tasa de conversión en comparación con el diseño de anuncio anterior (test group "psa").

In [36]:
# Usar np.where para asignar valores basados en la condición
df['tasa_conversion'] = np.where(df['test group'] == 'ad', ad_convertido, psa_convertido)

df

,user id,test group,converted,total ads,most ads day,most ads hour,tasa_conversion
0,1069124,ad,False,130,Monday,20,0.03
1,1119715,ad,False,93,Tuesday,22,0.03
2,1144181,ad,False,21,Tuesday,18,0.03
3,1435133,ad,False,355,Tuesday,10,0.03
4,1015700,ad,False,276,Friday,14,0.03
...,...,...,...,...,...,...,...
588096,1278437,ad,False,1,Tuesday,23,0.03
588097,1327975,ad,False,1,Tuesday,23,0.03
588098,1038442,ad,False,3,Tuesday,23,0.03
588099,1496395,ad,False,1,Tuesday,23,0.03


In [38]:
# Extraer las tasas de conversión para cada grupo
ad_conversion = df[df['test group'] == 'ad']['converted']
psa_conversion = df[df['test group'] == 'psa']['converted']

In [41]:
# Realizar la prueba t
t_stat, p_value = ttest_ind(ad_conversion, psa_conversion)
if p_value < 0.05:
    print(f"p-value = {p_value}: Rechazamos la hipótesis nula. Hay una diferencia significativa en las tasas de conversión entre los grupos 'ad' y 'psa'.")
else:
    print(f"p-value = {p_value}: No podemos rechazar la hipótesis nula. No hay evidencia suficiente para afirmar que hay una diferencia significativa.")

p-value = 1.7033052627831264e-13: Rechazamos la hipótesis nula. Hay una diferencia significativa en las tasas de conversión entre los grupos 'ad' y 'psa'.


Ejercicio 2: 
Comparar la cantidad promedio de anuncios vistos por los usuarios en los grupos "ad" y "psa" y determinar si hay una diferencia significativa entre ellos.

In [43]:

# Separar los datos por grupo
ad_ads_viewed = df[df['test group'] == 'ad']['total ads']
psa_ads_viewed = df[df['test group'] == 'psa']['total ads']

In [45]:
# Realizar la prueba t
t_stat, p_value = ttest_ind(ad_ads_viewed, psa_ads_viewed)

print(f"Prueba t - Valor p: {p_value}")

if p_value < 0.05:
    print(f"p-value = {p_value}: Rechazamos la hipótesis nula. Hay una diferencia significativa en la cantidad promedio de anuncios vistos por usuario entre los grupos 'ad' y 'psa'.")
else:
    print(f"p-value = {p_value}: No podemos rechazar la hipótesis nula. No hay evidencia suficiente para afirmar que hay una diferencia significativa.")

Prueba t - Valor p: 0.8306134186810267
p-value = 0.8306134186810267: No podemos rechazar la hipótesis nula. No hay evidencia suficiente para afirmar que hay una diferencia significativa.
